In [ ]:
!pip install g2pk

In [ ]:
from g2pk import G2p
import pandas as pd
import numpy as np
import random
import re
import os

- g2p 데이터 생성 코드

In [ ]:
# 데이터 불러오기
train = pd.read_csv("train.csv")

   
# 데이터 개수 계산
num_samples = len(train)

# 랜덤으로 7분의 1 선택
sample_indices = random.sample(range(num_samples), num_samples // 7)

# g2p 변환을 위한 객체 생성
g2p = G2p()

n1 = 0
n2 = 0
# 선택된 데이터에 대해 g2p 변환 적용 후 기존 데이터에 추가
for idx in sample_indices:
    sentence = train.loc[idx, 'text']
    target = train.loc[idx, 'target']
    
    # 50%의 확률로 descriptive 옵션 적용
    if random.random() < 0.5:
        converted_sentence = g2p(sentence, descriptive=True)
        n1 += 1
        print(converted_sentence," : descriptive, ",n1)
    else:
        converted_sentence = g2p(sentence)
        n2 += 1
        print(converted_sentence," : not_descriptive, ",n2)
    
    # 기존 데이터에 g2p로 변환된 문장과 타겟을 추가
    train = pd.concat([train, pd.DataFrame({'text': [converted_sentence], 'target': [target]})], ignore_index=True)

# 결과 확인
print(train)

# 결과를 새로운 CSV 파일로 저장
train.to_csv("new_total_arg_with_g2p.csv", index=False)


- g2p 영어 발음대로 읽어주는 기능 -> 잘 안됨

In [ ]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '../data')
OUTPUT_DIR = os.path.join(BASE_DIR, '../output')

In [ ]:
def extract_english_characters(data):
    data = str(data)
    english_pattern = re.compile(r'[a-zA-Z]')
    return ''.join(english_pattern.findall(data))

g2p = G2p()

# 데이터 불러오기
train1 = pd.read_csv(os.path.join(DATA_DIR, 'train_p2g_added.csv')) #1183개
train2 = pd.read_csv(os.path.join(DATA_DIR, 'train.csv')) #7000개
train = pd.concat([train2, train1], axis=0, ignore_index=True)
train = train.drop_duplicates(subset=['text']) #8183

train['extracted_english'] = train['text'].apply(extract_english_characters)

for index, row in train.iterrows():
    if row['extracted_english'] is not None and row['extracted_english'] != '':
        transliterated_text = g2p(row['extracted_english'])
        result_text = re.sub(row['extracted_english'], row['extracted_english']+"("+transliterated_text+")", row['text'])
        print(result_text)
        train = pd.concat([train, pd.DataFrame({'text': [converted_sentence], 'target': [target]})], ignore_index=True)

print(train[['text', 'text2']])
